Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = './notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [15]:
train_labels[:, None].shape

(200000, 1)

In [16]:
train_labels[:,None]

array([[4],
       [9],
       [6],
       ...,
       [2],
       [4],
       [4]], dtype=int32)

In [19]:
np.arange(10) == train_labels[0,None]

array([False, False, False, False,  True, False, False, False, False,
       False])

In [20]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [21]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

First we build the logistic model with L2 regularization

In [ ]:
# Logistic Model with SGD
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # Use placeholder to hold the place for input minibatch
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    regu_lamda = tf.placeholder(tf.float32)
    
    # Variables
    weights = tf.Variable(tf.truncated_normal([image_size*image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation
    logits = tf.matmul(tf_train_dataset, weights)+biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels)) + regu_lamda * tf.nn.l2_loss(weights)
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights)+biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights)+biases)

In [25]:
num_steps = 3001

with tf.Session(graph = graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), : ]
        batch_labels = train_labels[offset:(offset + batch_size), : ]
        
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels, regu_lamda : 1e-3}
        _, l, prediction = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f"%(step, l))
            print("Minibatch accuracy： %.1f%%"%(accuracy(prediction, batch_labels)))
            print("Validation accuracy: %.1f%%"%(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: %.1f%%"%(accuracy(test_prediction.eval(), test_labels)))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 18.814568
Minibatch accuracy： 16.4%
Validation accuracy: 13.6%
Minibatch loss at step 500: 2.782284
Minibatch accuracy： 79.7%
Validation accuracy: 75.9%
Minibatch loss at step 1000: 1.966941
Minibatch accuracy： 75.0%
Validation accuracy: 78.7%
Minibatch loss at step 1500: 1.536669
Minibatch accuracy： 76.6%
Validation accuracy: 79.8%
Minibatch loss at step 2000: 1.000396
Minibatch accuracy： 82.0%
Validation accuracy: 80.8%
Minibatch loss at step 2500: 1.034003
Minibatch accuracy： 71.9%
Validation accuracy: 80.6%
Minibatch loss at step 3000: 0.721865
Minibatch accuracy： 85.2%
Validation accuracy: 81.9%
Test accuracy: 88.1%


It looks better than the logistic model without l2 regularization in the previous assignment, let's see will it work in 1-hidden layer neural network

In [36]:
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    # Set the input minibatch data
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    regu_lamda = tf.placeholder(tf.float32)
    
    # Variables
    weights = [
        tf.Variable(tf.truncated_normal([image_size*image_size, num_hidden_nodes])),
        tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
    ]
    biases = [
        tf.Variable(tf.zeros([num_hidden_nodes])),
        tf.Variable(tf.zeros([num_labels]))
    ]
    
    # Training computation
    hidden_layer_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights[0]) + biases[0])
    logits = tf.matmul(hidden_layer_train, weights[1]) + biases[1]
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) + \
            regu_lamda * (tf.nn.l2_loss(weights[0]) + tf.nn.l2_loss(weights[1]))
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions
    train_prediction = tf.nn.softmax(logits)
    hidden_layer_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights[0])+biases[0])
    valid_prediction = tf.nn.softmax(tf.matmul(hidden_layer_valid, weights[1]) + biases[1])
    hidden_layer_test = tf.nn.relu(tf.matmul(tf_test_dataset,weights[0])+biases[0])
    test_prediction = tf.nn.softmax(tf.matmul(hidden_layer_test, weights[1]) + biases[1])

In [38]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels, regu_lamda: 1e-3}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 629.675781
Minibatch accuracy: 13.3%
Validation accuracy: 38.7%
Minibatch loss at step 500: 207.511551
Minibatch accuracy: 82.8%
Validation accuracy: 78.7%
Minibatch loss at step 1000: 120.824661
Minibatch accuracy: 81.2%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 71.829987
Minibatch accuracy: 78.1%
Validation accuracy: 80.2%
Minibatch loss at step 2000: 41.475979
Minibatch accuracy: 87.5%
Validation accuracy: 85.5%
Minibatch loss at step 2500: 25.319036
Minibatch accuracy: 83.6%
Validation accuracy: 86.5%
Minibatch loss at step 3000: 15.480872
Minibatch accuracy: 91.4%
Validation accuracy: 87.4%
Test accuracy: 92.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [39]:
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    # Set the input minibatch data
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    regu_lamda = tf.placeholder(tf.float32)
    
    # Variables
    weights = [
        tf.Variable(tf.truncated_normal([image_size*image_size, num_hidden_nodes])),
        tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
    ]
    biases = [
        tf.Variable(tf.zeros([num_hidden_nodes])),
        tf.Variable(tf.zeros([num_labels]))
    ]
    
    # Training computation
    hidden_layer_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights[0]) + biases[0])
    logits = tf.matmul(hidden_layer_train, weights[1]) + biases[1]
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) + \
            regu_lamda * (tf.nn.l2_loss(weights[0]) + tf.nn.l2_loss(weights[1]))
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions
    train_prediction = tf.nn.softmax(logits)
    hidden_layer_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights[0])+biases[0])
    valid_prediction = tf.nn.softmax(tf.matmul(hidden_layer_valid, weights[1]) + biases[1])
    hidden_layer_test = tf.nn.relu(tf.matmul(tf_test_dataset,weights[0])+biases[0])
    test_prediction = tf.nn.softmax(tf.matmul(hidden_layer_test, weights[1]) + biases[1])

In [40]:
num_steps = 3001
num_batches = 5

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = ((step % num_batches) * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels, regu_lamda: 1e-3}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 675.561157
Minibatch accuracy: 12.5%
Validation accuracy: 28.5%
Minibatch loss at step 500: 190.249329
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 1000: 115.377594
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 1500: 69.971329
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 2000: 42.434574
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 2500: 25.735155
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 3000: 15.608627
Minibatch accuracy: 100.0%
Validation accuracy: 77.5%
Test accuracy: 84.2%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [60]:
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    # Set the input minibatch data
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    regu_lamda = tf.placeholder(tf.float32)
    
    # Variables
    weights = [
        tf.Variable(tf.truncated_normal([image_size*image_size, num_hidden_nodes])),
        tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
    ]
    biases = [
        tf.Variable(tf.zeros([num_hidden_nodes])),
        tf.Variable(tf.zeros([num_labels]))
    ]
    
    # Training computation
    hidden_layer_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights[0]) + biases[0])
    hidden_layer_train = tf.nn.dropout(hidden_layer_train, 0.5)
    logits = tf.matmul(hidden_layer_train, weights[1]) + biases[1]
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) + \
            regu_lamda * (tf.nn.l2_loss(weights[0]) + tf.nn.l2_loss(weights[1]))
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions
    train_prediction = tf.nn.softmax(logits)
    hidden_layer_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights[0])+biases[0])
    valid_prediction = tf.nn.softmax(tf.matmul(hidden_layer_valid, weights[1]) + biases[1])
    hidden_layer_test = tf.nn.relu(tf.matmul(tf_test_dataset,weights[0])+biases[0])
    test_prediction = tf.nn.softmax(tf.matmul(hidden_layer_test, weights[1]) + biases[1])

In [61]:
num_steps = 3001
num_batches = 5

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = ((step % num_batches) * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels, regu_lamda: 1e-3}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 779.323364
Minibatch accuracy: 10.9%
Validation accuracy: 26.0%
Minibatch loss at step 500: 191.949600
Minibatch accuracy: 100.0%
Validation accuracy: 78.6%
Minibatch loss at step 1000: 116.483948
Minibatch accuracy: 100.0%
Validation accuracy: 78.3%
Minibatch loss at step 1500: 70.666283
Minibatch accuracy: 100.0%
Validation accuracy: 78.6%
Minibatch loss at step 2000: 42.866856
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Minibatch loss at step 2500: 26.000044
Minibatch accuracy: 100.0%
Validation accuracy: 78.9%
Minibatch loss at step 3000: 15.768623
Minibatch accuracy: 100.0%
Validation accuracy: 79.1%
Test accuracy: 85.8%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [71]:
batch_size = 128
num_hidden1_nodes = 1024
num_hidden2_nodes = 512
num_hidden3_nodes = 256

graph = tf.Graph()
with graph.as_default():
    # Set the input minibatch data
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    regu_lamda = tf.placeholder(tf.float32)
    global_step = tf.Variable(0)
    
    # Variables
    weights = [
        tf.Variable(tf.truncated_normal([image_size*image_size, num_hidden1_nodes],
                                       stddev=np.sqrt(2.0 / (image_size * image_size)))),
        tf.Variable(tf.truncated_normal([num_hidden1_nodes, num_hidden2_nodes],
                                       stddev=np.sqrt(2.0 / num_hidden1_nodes))),
        tf.Variable(tf.truncated_normal([num_hidden2_nodes, num_hidden3_nodes],
                                       stddev=np.sqrt(2.0 / num_hidden2_nodes))),
        tf.Variable(tf.truncated_normal([num_hidden3_nodes, num_labels],
                                       stddev=np.sqrt(2.0 / num_hidden3_nodes)))
    ]  # If u don't initialize the parameters with adaptive stddev, the loss maybe NaN
    
    biases = [
        tf.Variable(tf.zeros([num_hidden1_nodes])),
        tf.Variable(tf.zeros([num_hidden2_nodes])),
        tf.Variable(tf.zeros([num_hidden3_nodes])),       
        tf.Variable(tf.zeros([num_labels]))
    ]
    
    # Training computation
    hidden_layer1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights[0]) + biases[0])
    #hidden_layer1_train = tf.nn.dropout(hidden_layer1_train, 0.5)
    hidden_layer2_train = tf.nn.relu(tf.matmul(hidden_layer1_train, weights[1]) + biases[1])
    #hidden_layer2_train = tf.nn.dropout(hidden_layer2_train, 0.5)
    hidden_layer3_train = tf.nn.relu(tf.matmul(hidden_layer2_train, weights[2]) + biases[2])
    #hidden_layer3_train = tf.nn.dropout(hidden_layer3_train, 0.5)
    logits = tf.matmul(hidden_layer3_train, weights[3]) + biases[3]
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) + \
            regu_lamda * (tf.nn.l2_loss(weights[0]) + tf.nn.l2_loss(weights[1]) + tf.nn.l2_loss(weights[2]) + \
                             tf.nn.l2_loss(weights[3]))
    
    # Optimizer
    learning_rate = tf.train.exponential_decay(0.5, global_step, 500, 0.8, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions
    train_prediction = tf.nn.softmax(logits)
    hidden_layer1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights[0])+biases[0])
    hidden_layer2_valid = tf.nn.relu(tf.matmul(hidden_layer1_valid, weights[1])+biases[1])
    hidden_layer3_valid = tf.nn.relu(tf.matmul(hidden_layer2_valid, weights[2])+biases[2])
    valid_prediction = tf.nn.softmax(tf.matmul(hidden_layer3_valid, weights[3]) + biases[3])
    hidden_layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset,weights[0])+biases[0])
    hidden_layer2_test = tf.nn.relu(tf.matmul(hidden_layer1_test, weights[1])+biases[1])
    hidden_layer3_test = tf.nn.relu(tf.matmul(hidden_layer2_test, weights[2])+biases[2])
    test_prediction = tf.nn.softmax(tf.matmul(hidden_layer3_test, weights[3]) + biases[3])

In [72]:
num_steps = 10000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels, regu_lamda: 1e-3}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.804791
Minibatch accuracy: 10.2%
Validation accuracy: 39.5%
Minibatch loss at step 500: 1.374182
Minibatch accuracy: 87.5%
Validation accuracy: 86.1%
Minibatch loss at step 1000: 1.106662
Minibatch accuracy: 86.7%
Validation accuracy: 87.8%
Minibatch loss at step 1500: 0.999305
Minibatch accuracy: 80.5%
Validation accuracy: 88.5%
Minibatch loss at step 2000: 0.682179
Minibatch accuracy: 93.0%
Validation accuracy: 89.0%
Minibatch loss at step 2500: 0.716244
Minibatch accuracy: 87.5%
Validation accuracy: 89.5%
Minibatch loss at step 3000: 0.667718
Minibatch accuracy: 89.1%
Validation accuracy: 89.9%
Minibatch loss at step 3500: 0.601104
Minibatch accuracy: 91.4%
Validation accuracy: 89.8%
Minibatch loss at step 4000: 0.598992
Minibatch accuracy: 91.4%
Validation accuracy: 90.3%
Minibatch loss at step 4500: 0.468947
Minibatch accuracy: 95.3%
Validation accuracy: 90.4%
Minibatch loss at step 5000: 0.529402
Minibatch accuracy: 92.2%
Validation accurac